<a href="https://colab.research.google.com/github/bharathbolla/EIP4/blob/assignment_4/EVA6_%2B_END_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun May 23 11:12:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import print_function

import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
from torch.utils.data import Dataset, TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from random import randint

torch.set_printoptions(linewidth=120)

# Setting up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print("The device available is: ", device)

The device available is:  cuda:0


## Combined Dataset

In [3]:
def to_categorical(y,num_classes=10):
    """ 1-hot encodes a tensor """
    return (np.eye(num_classes, dtype='uint8')[y])
def cat_array(x):
  return torch.from_numpy(np.array([to_categorical(xi) for xi in x]) )

In [4]:
# Create a class to combine MNIST dataset and random numbers between 0 and 9
class Combined_Dataset():

  # We pass the train variable to get train or test data, and batch_size
  def __init__(self, train, batch_size):

      self.batch_size = batch_size
      # Load the MNIST data into the data_loader object
      self.data_loader = torch.utils.data.DataLoader(
          torchvision.datasets.MNIST('/files/', train=train, download=True,
                                transform=torchvision.transforms.Compose([
                                  torchvision.transforms.ToTensor(),
                                  torchvision.transforms.Normalize(
                                    (0.1307,), (0.3081,))
                                ])),
          batch_size=self.batch_size, shuffle=True)

      # Number of samples in the dataaset
      self.dataset = self.data_loader.dataset            

  # getitem function creats batches of our dataset on the fly by calling next(iter())
  def __getitem__(self, index):
      # Extract one batch of the MNIST data_loader
      image, label = next(iter(self.data_loader))

      # Generate randoms numbers between 0 and 9 of size=batch_size. The datatype is float as this is the input required for the network
      random_numbers = torch.tensor([randint(0,9) for _ in range(self.batch_size)], dtype=torch.int8)
      print(random_numbers.shape)
      np_random_numbers = random_numbers.numpy()
      print("NUmpy Random Numbers", np_random_numbers.shape, type(np_random_numbers))
      # Combine inputs and outputs as a list after transfering the tensors to the GPU
      cat_rand_array = cat_array(np_random_numbers)
      print("Cat Random Numbers Input shape", cat_rand_array.shape)
      x = [image.to(device), cat_rand_array.to(device)]
      print("Random Numbers Input shape", x[1].shape)
      #x = [image.to(device), random_numbers.to(device)]
      # y labels for addition of number is reshaped to [32,1] as MSE requires it in this format
      #y = [label.to(device), (label+random_numbers).reshape([32,1]).to(device)]
      y = [label.to(device), (label+random_numbers).to(device)]
      #print(y[0],y[1])


      return x, y

  def __len__(self):
      return len(self.data_loader)

# Set the batch size
batch_size = 32

# Create the train and test dataset
train_data = Combined_Dataset(train=True, batch_size=batch_size)
test_data = Combined_Dataset(train=False, batch_size=batch_size)


Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw




Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw




Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw




Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
print(f"Number of train batches: {len(train_data)}")
print(f"Number of test batches: {len(test_data)}")

print(f"Number of train samples: {len(train_data.dataset)}")
print(f"Number of test samples: {len(test_data.dataset)}")

x, y = next(iter(train_data))

print(f"Shape of input data is: [{x[0].shape}, {x[1].shape}]")
print(f"Shape of output data is: [{y[0].shape}, {y[1].shape}]")

Number of train batches: 1875
Number of test batches: 313
Number of train samples: 60000
Number of test samples: 10000
torch.Size([32])
NUmpy Random Numbers (32,) <class 'numpy.ndarray'>
Cat Random Numbers Input shape torch.Size([32, 10])
Random Numbers Input shape torch.Size([32, 10])
Shape of input data is: [torch.Size([32, 1, 28, 28]), torch.Size([32, 10])]
Shape of output data is: [torch.Size([32]), torch.Size([32])]


## Model

In [6]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv1 = nn.Conv2d(1,32, 3,padding=0,bias=False )
    self.conv2 = nn.Conv2d(32,16, 3,padding=0,bias=False )
    self.conv3 = nn.Conv2d(16, 8,3, padding=0, bias=False)
    self.linear1 = nn.Linear(200, 16)
    self.linear2 = nn.Linear(16,10)
    self.linear3 = nn.Linear(20,10)
    self.flatten = nn.Flatten()

  def forward(self, X, Y):
    X = F.relu(self.conv1(X))
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2)
    X = F.relu(self.conv3(X))
    X = F.max_pool2d(X, 2)
    X = self.flatten(X)
    X = F.relu(self.linear1(X))
    X = self.linear2(X)
    print("X shape before X1 squeeze", X.shape)
    X1 =  torch.squeeze(X)
    print("X1 shape after X1 squeeze", X.shape, type(X))
    #Y = to_categorical(Y)
    print("Y Shape", Y.shape, type(Y))
    Y = Y.type(torch.cuda.FloatTensor)
    Y = torch.cat((X,Y),1) # This will result in 20 dimensions
    print("Y Shape after concat", Y.shape, type(Y))
    #Y = Y.to(device)
    Y = self.linear3(Y)
    print("Shape of X", X.shape)
    print("Shape of Y", Y.shape)
    #output1 = F.log_softmax(X)
    #print("oUTPUT1 sHAPE", output1.shape)
    #output2 = torch.unsqueeze(Y,0)
    #output2 = F.log_softmax(output2)
    #print("Output2 Shape", output2.shape)
    return X, Y

In [ ]:
!pip install torchsummary

In [7]:
# Setting up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print("The device available is: ", device)

The device available is:  cuda:0


In [8]:
# Create an object of the class Network and transfer it to the GPU
model = Model().to(device)
print(" The model layers are: ")
print(model)

print("\nShape of parameters in each layer is: ")
for name, param in model.named_parameters():
    print(name, '\t\t', param.shape)

# Selecting the loss function and optimizer  for the model
CE_loss = nn.CrossEntropyLoss()
MSE_loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

 The model layers are: 
Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv3): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (linear1): Linear(in_features=200, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=10, bias=True)
  (linear3): Linear(in_features=20, out_features=10, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

Shape of parameters in each layer is: 
conv1.weight 		 torch.Size([32, 1, 3, 3])
conv2.weight 		 torch.Size([16, 32, 3, 3])
conv3.weight 		 torch.Size([8, 16, 3, 3])
linear1.weight 		 torch.Size([16, 200])
linear1.bias 		 torch.Size([16])
linear2.weight 		 torch.Size([10, 16])
linear2.bias 		 torch.Size([10])
linear3.weight 		 torch.Size([10, 20])
linear3.bias 		 torch.Size([10])


In [9]:
for epoch in range(1):  # Loop over the dataset multiple times

    total_loss = 0.0
    total_correct_1, total_correct_2 = 0, 0
    # Loop over the entire length of train data
    for i in range(len(train_data)):
        # Get the inputs and outputs
        # Input data x is a list of [images, random numbers], output data y is a list of [classes, sum of numbers]
        x, y = next(iter(train_data))

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + Backward + Optimize
        output1, output2 = model(x[0], x[1])
        print("Output1", output1)
        print("Output2", output2)
        # Use the CE loss for classification and MSE loss for addition 
        #loss = CE_loss(output1, y[0]) + CE_loss(output2, y[1])
        loss = CE_loss(output1, y[0])
        #loss = CE_loss(output2, y[1])
        loss.backward()
        optimizer.step()

        # Calculate statistics
        total_loss += loss.item()
        #total_correct_1 += output1.argmax(dim=1).eq(y[0]).sum().item()
        #total_correct_2 += (torch.round(output2) == torch.round(y[1])).sum().item()
        
    # Print statistics        
    #print(f"Epoch: {epoch+1}, loss: {total_loss}, Classification Acc: {100 * (total_correct_1/(len(train_data.dataset)))}, Addition Acc: {100 * (total_correct_2/(len(train_data.dataset)))}")
    #print(f"Epoch: {epoch+1}, loss: {total_loss}, Classification Acc: {100 * (total_correct_1/(len(train_data.dataset)))}")
    
print('Finished Training')

Streaming output truncated to the last 5000 lines.
        [ 1.9513e+00, -3.4251e+00,  1.4801e+00,  3.0557e+00, -1.8107e+00,  5.8521e+00, -6.2959e+00, -2.8303e+00,
          1.9775e+01,  9.3524e+00],
        [-2.0412e+00, -4.8471e+00,  2.4090e+00, -1.1767e+00, -7.2720e-01,  1.0521e+01,  3.6497e+00, -4.7297e+00,
          9.3516e+00,  4.1367e-01],
        [-3.6741e+00, -9.2667e-01, -5.7562e+00, -2.2187e+00,  1.0311e+01,  2.1889e+00, -1.6831e-01, -3.1962e+00,
         -1.1586e+00,  1.9247e+00],
        [-3.0405e+00, -5.0925e-01,  6.5285e+00,  1.8175e+01, -9.6986e+00,  5.0661e+00, -1.0586e+01,  4.1817e+00,
          5.1105e+00,  3.6529e+00],
        [-2.2674e+00, -1.2107e+00,  7.1250e-01, -6.0249e+00,  1.3914e+01, -7.7310e-01, -4.0506e-01, -2.5617e+00,
          1.7776e+00, -4.2226e-01]], device='cuda:0', grad_fn=<AddmmBackward>)
Output2 tensor([[ 0.4418, -0.9626, -1.3823, -1.4622, -0.5315,  1.1539, -0.8773,  0.0103,  0.3292,  0.3323],
        [ 2.4219,  2.5882, -3.4232, -2.7885,  4.9130,